<p style="font-size:40px; font-weight:bold">Voila 4 Analysts</p>
<br>

# Introduction

Voila is a tool to turn a Jupyter notebook into an interactive dashboard as a standalone web application.  Voila helps notebook authors share their work with others who don’t need to see the code or execute the notebook manually.  You can read more about Voila on the [Jupyter Blog](https://blog.jupyter.org/and-voil%C3%A0-f6a2c08a4a93 "Voila on the Jupyter Blog") (warning: possibly more technical detail than you want) and check out some examples at the official [Voila Gallery](https://voila-gallery.org/ "Voila Gallery").

This guide is designed to help you write notebooks that work in Voila.  It’s primarily aimed at authors who use Voila in a managed environment where Jupyter is already installed for you, so it doesn’t cover installing or configuring Voila -- see the [official documentation](https://voila.readthedocs.io "Voila documentation") for that.  However, we will mention a few configuration settings that may be of interest, so we’ll use the term “Voila administrator” to refer to the person or team that manages your environment if you as the notebook author don’t have control over it yourself.

If you’re writing a notebook and want to try it out in Voila, you’ll need to have the Voila preview extension installed in your Jupyter environment.  If you’re using the classic notebook UI, look for the Voila button, which will pop open a new browser tab and run the notebook in Voila.

![Voila button in Jupyter Classic](screenshots/classic_voila_button.png)

If you’re using Jupyter Lab, look for the Voila preview -- it's the circle icon on the far right of the screenshot below, and it has a tooltip of "Render with Voila" when you mouse-over it.  It will open a new tab within Lab next to your notebook, and you can have it refresh whenever you save your notebook.  Voila is also available under the View menu in Lab.

![Voila button in Jupyter Lab](screenshots/lab_voila_button.png)

Depending on the hosting options available in your organization, you may also be able to deploy your notebook as a standalone Voila web application.

In the rest of this guide, we'll look at how Voila is different from Jupyter and some techniques to help transition your notebooks into dashboards.

Note: if you're *not* launching this guide in binder, you may want to use the [Setup Notebook](V4A%20Setup.ipynb "Setup Notebook") to install the packages used in the examples.

# Voila Execution Model

The most important thing to know about Voila is that it executes notebooks a little differently than Jupyter itself, so there are a few things you can do in Jupyter that won’t work in Voila.  You can think of Voila’s execution model as running the notebook in two phases: the Rendering Phase and the Interactive Phase.  (These aren't official Voila terminology, just something we'll use here to help talk about when things are happening.)

During the Rendering Phase, Voila basically does a “Run All” to execute all the code cells in your notebook and capture the output, as well as to render any markdown cells.  (If you’ve ever exported a notebook to HTML, it’s essentially the same process.)  This output from the Rendering Phase is what the user will see when they first load your noteboook as a dashboard.

During the Interactive Phase, the user can manipulate widgets to affect what the dashboard displays.  The notebook’s kernel is still alive during the Interactive Phase, but the key thing to remember is that *only widgets can communicate with the kernel* -- you can no longer re-run code cells or display new output!

So what does this mean when writing a notebook?  In short, your notebook needs to be Run-All friendly, and any new output generated during the Interactive Phase must be displayed using widgets that were created and displayed during the Rendering Phase.  We’ll get into more detail with some examples as we go.

# Run-All Friendliness

The main thing to remember about the Rendering Phase is that Voila will run your notebook from top to bottom.  Most importantly, that means you can’t do anything that will block the notebook’s execution while waiting for user input.  Unfortunately, that means that some otherwise useful tools like Python’s built-in [`input()`](https://docs.python.org/3/library/functions.html#input "Python input documentation") and [`getpass()`](https://docs.python.org/3/library/getpass.html#getpass.getpass "Python getpass documentation") and libraries like [ipython_blocking](https://github.com/kafonek/ipython_blocking "ipython_blocking library") must be avoided in Voila.  Instead, you’ll generally need to use [ipywidgets](https://ipywidgets.readthedocs.io "ipywidgets documentation") to get user input -- and if you do need to wait for that input before you can do anything, you may need to restructure your notebook.  We haven't talked about everything in this example yet, but if you want to get an idea of how you might need to restructure your notebook, check out [Example 1](V4A%20Example%2001%20-%20Username%20and%20Password.ipynb "Example 1").

A secondary consideration is that you want the notebook execution to complete *quickly*.  Voila has an *optional* cell execution timeout (configurable by your Voila administrator), but from a practical standpoint you probably don’t want your users waiting very long for the dashboard to load.  So, if you have a lengthy computation, it may be better to defer that to the Interactive Phase where it could be triggered by a user action like clicking a button.

# Widget Interaction

Many notebooks will work in Voila without any changes.  If the notebook can compute everything it needs and generate all the outputs during the Rendering Phase, then you probably don’t need to change anything.  This doesn’t necessarily mean that your notebook isn’t interactive -- there are a number of chart and map libraries that generate outputs with interactive features built into the outputs themselves.

Things start to get interesting when your notebook’s input data requires parameters from the user.  In these situations, you’ll probably use [ipywidgets](https://ipywidgets.readthedocs.io "ipywidgets documentation").  Our goal here is to focus on making things work in Voila, so if you haven’t used ipywidgets before, there’s an [introduction](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html "Widget basics") in the documentation, and [this article](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916 "Interactive Widgets") is a good overview as well.

Many simple use cases with ipywidgets will work fine in Voila.  In particular, if you can wrap your notebook’s main analysis into a single function call, then the `interact` family of functions provides a simple way to attach a widget user interface that will work in Voila.  The widgets will be created during the Rendering Phase, and your analysis function will be called on demand during the Interactive Phase.  You can read more about these functions in the [documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html "Using Interact"), but here’s a quick summary of the ones you’re likely to use:

 * `interact` - This will *automatically* generate widgets based on the parameters to your function.  Whenever the user changes one of the widgets, your function will get called with the new parameters, and the output will be displayed.
 * `interact_manual` - Use this to make the user click a submit button to call your function.  This is a good idea if your function takes a few seconds to run and you don’t want it to be called for every little adjustment of the input widgets.
 * `interactive_output` - Use this if you want more control over the layout of the widget UI using [container widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Container/Layout-widgets "Container/Layout widgets").

Here's an [example](V4A%20Example%2002%20-%20Interact.ipynb "Example 2") of these functions in action.

# Widget Events

More complicated widget user interfaces may rely on [widget events](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html "Widget Events").  This feature enables you to attach callback functions that will be executed when the value of a widget changes (`observe`) or a button is clicked (`on_click`).  There are many scenarios in which you might want to use widget events and callbacks; for example:

 * The `interact` family isn’t a good fit, perhaps because your analysis is too complex to wrap into a single function call, or you need more control over the input widgets.
 * As we mentioned earlier, your analysis might be too slow to execute during Voila’s Rendering Phase, so you want to trigger it with a button click.
 * You have a multi-stage analysis, where the user needs to set input parameters at each stage based on the result of the previous stage.
 * You need some initial input from the user before you can even generate the rest of the UI.

With `interact`, we still had a clean separation between the Rendering Phase and the Interactive Phase. However, in these scenarios, it starts to get more complicated, and you have to be more careful about what works in Voila and what does not.

## The Output Widget

We mentioned earlier that once you get to the Interactive Phase, the only way for your notebook to generate new output is by using widgets that were created during the Rendering Phase.  For example, suppose you have a [Button](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Button "Button Widget") widget with an `on_click` callback function attached that will run your analysis:

```python
from IPython.display import display
b = widgets.Button(description='click me')
def callback(w):
    print('callback')
b.on_click(callback)
display(b)
```

If that callback function uses `print()` or functions from [`IPython.display`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html "IPython.display documentation"), **that output will not show up in Voila** because the kernel is no longer connected to the output areas of your code cells, only to widgets.

This is where the Output widget comes in.  The Output widget hooks into Jupyter’s display system to capture any output and display it as part of the widget state.  (It’s actually how the `interact` functions work -- in fact, `interactive_output` returns an Output widget.)  You can make your callback function work in Voila with three easy steps:

 1. Create an Output widget.
 2. Add a decorator to your callback function to have the Output widget capture any output the function generates.
 3. Place the Output widget somewhere during the Rendering Phase.  You can do this using `IPython.display` to draw it, by including it in a layout built with container widgets (`Box`, etc.), or simply by referencing it as the last thing in a cell.

Here’s how to update the example from above:

```python
from IPython.display import display
b = widgets.Button(description='click me')
out = widgets.Output()

@out.capture()
def callback(w):
    print('callback')
b.on_click(callback)
display(b, out)
```

You can try it out in [Example 3](V4A%20Example%2003%20-%20Output%20Widgets.ipynb "Example 3") and read more about the Output widget in the [documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html "Output Widget").

## Wrapper Widgets

What if you need to generate a plot from inside your callback function?  In many cases, you can probably use an Output widget and `IPython.display`, but several libraries have additional wrappers that provide better integration with the rest of ipywidgets.  You may not need them in if you’re just running your notebook in Jupyter itself, but they can add value when building dashboards for Voila.  Here are a few of these wrapper widgets:

 * [`ipympl`](https://github.com/matplotlib/ipympl "ipympl library"), a separate library for wrapping matplotlib plots into a widget.  See [Example 4](V4A%20Example%2004%20-%20ipympl.ipynb "Example 4").
 * [`jupyter_bokeh`](https://github.com/bokeh/jupyter_bokeh "jupyter_bokeh library"), a separate library for wrapping Bokeh plots.  See [Example 5](V4A%20Example%2005%20-%20Bokeh.ipynb "Example 5").
 * [`FigureWidget`](https://plotly.com/python/figurewidget/ "Plotly FigureWidget documentation"), part of the Plotly library.  See [Example 6](V4A%20Example%2006%20-%20Plotly.ipynb "Example 6").

## Modifying Widget Layouts

Sometimes you may not be able to pre-position all the widgets you need during the Rendering Phase and you need to modify your UI on the fly during the Interactive Phase.  For example, maybe you need the user to provide login information for a remote server before you can build the rest of your UI, or your analysis has multiple stages and you can’t generate the input widgets for stage 2 until you have the results from stage 1.  There are a number of ways to make this work; here are a few strategies we’ve seen:

 * Use a container layout widget - as long as the container is placed during the Rendering Phase, you can add and remove other widgets as children during the Interactive Phase.
 * If you have a good idea of all the possible widgets you’ll need, you can just create and place them all during the Rendering Phase, then toggle their visibility during the Interactive Phase.
 * The Output widget can display other widgets as well, so you can place an Output during the Rendering Phase, then create new widgets during the Interactive Phase and add them to the UI by using IPython.display captured by the Output widget.

Check out [Example 7](V4A%20Example%2007%20-%20Modifying%20Widget%20Layouts.ipynb "Example 7") for a quick demo of these strategies.

# Other Topics

This section contains an assortment of things we've encountered while converting notebooks into Voila dashboards.

## Output Files

Many notebooks generate output files that the user may want to download -- for example, you might export a pandas dataframe to csv or a map to kml.  There are two gotchas to keep in mind when generating output files in Voila.

First, Voila has a stricter security model than Jupyter itself and won’t let you download files unless they’ve been specifically allowed by the configuration.  By default, only a handful of image types are allowed, so you may have to contact your Voila administrator if this is a common use case.  The list of allowed files is configured using a list of regular expressions (see [documentation](https://voila.readthedocs.io/en/stable/customize.html#serving-static-files "Serving Static Files")).  One suggestion is for your administrator to add `'public.*'` to the list of expressions; that will allow any file type but requires you as the notebook author to explicitly opt in by having your filename start with `public`.

Second, you’ll probably want a clickable HTML link to the file, so you need to know the path to the file.  The absolute path will be different in Jupyter vs Voila, so we highly recommend using a [`FileLink`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.FileLink "FileLink documentation") with a relative path.

If you’re having trouble getting an output file to work, how do you know which gotcha is the problem?  If you’re getting a 403 (Forbidden), you’re getting blocked by the allowed file configuration.  If you’re getting a 404 (Not Found), you probably have the path to the file wrong.

Check out [Example 8](V4A%20Example%2008%20-%20Output%20Files.ipynb "Example 8") to see this in action.

## Are You in Voila?

Sometimes you may want your notebook to behave differently in Voila than it does in Jupyter.  For example, you might want to display intermediate output while running in Jupyter, but only show the final result when running as a Voila dashboard.  While it's probably best not to make heavy use of this feature, it is indeed possible.  As of Voila version 0.2, the environment variable `SERVER_SOFTWARE` will be set to `voila/{version}` when your notebook is running in Voila.  Here's a short [example](V4A%20Example%2009%20-%20Are%20You%20in%20Voila.ipynb "Example 9").

# When Things Go Wrong

There are a few ways that notebooks can fail when running in Voila.  There are some Voila server configuration options that may affect what you see, but this section will help you figure out what's going on.

## Cell Execution Errors

The most likely problem you'll see is an exception raised by one of the code cells in the notebook.  If the exception occurs during the Rendering Phase, you will see an error message in the dashboard.  However, by default, you will not see the usual Python exception message, but instead a message like this:

```
There was an error when executing cell [3]. Please run Voilà with --debug to see the error message.
```

Exception messages can potentially reveal sensitive code or data, so this default behavior is part of Voila's security model.  Your Voila administrator may be able to get more detail from server log files.  Depending on your environment, your administrator may also be able to run Voila in debug mode, in which case you will see the usual Python exception message.

If a cell raises an exception from a widget callback during the Interactive Phase, you may not see the error at all unless the callback is being captured by an Output widget.  You may need to at least temporarily add in an Output widget to help you debug.

## Cell Execution Timeouts

While the latest version of Voila does not have an execution timeout by default, your Voila administrator may have configured one.  If the Voila "executing" spinner lasts for a very long time, it's possible that your notebook is hung indefinitely for some reason.  If your administrator has configured a timeout, the notebook execution will get interrupted and you'll see a message like this:

```
Cell execution timed out, aborting notebook execution. Please run Voilà with --VoilaExecutor.interrupt_on_timeout=True to continue executing the rest of the notebook.
```

The configuration setting mentioned there can be set by your administrator to enable notebooks to keep going after a timeout.  However, we do *not* recommend using that option, because your notebook could end up in an undetermined state that leads to more errors at a later point.

## Javascript Problems

If you're using a visualization library and it displays properly in Jupyter but not Voila, there may be Javascript issues at play.  One possibility is that the library requires a Jupyter widget extension that may not be getting loaded in Voila; check with your administrator and reference [this section](https://voila.readthedocs.io/en/stable/using.html#using-third-party-widgets-with-voila "Third-party widgets with Voila") in the Voila documentation.  Another possibility is that the library isn't completely compatible with Voila.  Voila loads some of the core Javascript libraries in a different way than Jupyter itself, and we've occasionally seen that cause a problem.

## Server Errors

While extremely rare, if you see a 500 error, that means something went wrong in Voila itself while executing your notebook.  This may or may not have been caused by your notebook, and you will probably need your administrator to check the logs on the server to help you debug.

# Examples

Here is a list of the examples in Voila 4 Analysts.

 1. [Username and Password](V4A%20Example%2001%20-%20Username%20and%20Password.ipynb "Username and Password")
 2. [Interact](V4A%20Example%2002%20-%20Interact.ipynb "Interact")
 3. [Output Widgets](V4A%20Example%2003%20-%20Output%20Widgets.ipynb "Output Widgets")
 4. [ipympl](V4A%20Example%2004%20-%20ipympl.ipynb "ipympl")
 5. [Bokeh](V4A%20Example%2005%20-%20Bokeh.ipynb "Bokeh")
 6. [Plotly](V4A%20Example%2006%20-%20Plotly.ipynb "Plotly")
 7. [Modifying Widget Layouts](V4A%20Example%2007%20-%20Modifying%20Widget%20Layouts.ipynb "Modifying Widget Layouts")
 8. [Output Files](V4A%20Example%2008%20-%20Output%20Files.ipynb "Output Files")
 9. [Are You in Voila?](V4A%20Example%2009%20-%20Are%20You%20in%20Voila.ipynb "Are You in Voila?")

For more examples, check out the [Voila Gallery](https://voila-gallery.org/ "Voila Gallery") and the [Voila GitHub](https://github.com/voila-dashboards/voila/tree/master/notebooks "Voila GitHub examples").


# Documentation

 * [Voila documentation](https://voila.readthedocs.io/en/stable/)
 * [Voila GitHub](https://github.com/voila-dashboards/voila "Voila GitHub")
 * [ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/index.html "ipywidgets")
 * [IPython.display documentation](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html "IPython.display documentation")